In [1]:
%load_ext autotime
import pyspark
from currency_converter import CurrencyConverter, ECB_URL
from delta import *
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "16g") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.driver.extraJavaOptions", "-Xss4M")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/05/11 09:32:36 WARN Utils: Your hostname, Andreis-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.145 instead (on interface en0)
23/05/11 09:32:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/andreikirpichev/.ivy2/cache
The jars for the packages stored in: /Users/andreikirpichev/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d498460-aa18-412e-a111-5fce3ed915ff;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 110ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number

23/05/11 09:32:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


time: 3.07 s (started: 2023-05-11 09:32:35 +02:00)


In [3]:
bm = spark.read.format("delta").load("/Users/andreikirpichev/parquet/betmarkets_lifespan/")
bm_contest = bm.withColumn('contest_id', split(bm.contest_id, '\|').getItem(1))
bm_football = bm_contest.filter(bm_contest["sport"] == "Football")
# test: from 2022-10-18 09:32:32.232000  to  2022-10-28 14:23:44.972000 (974330, 13)
MAX_START = "2022-10-28 14:23:42.051000"
MIN_END = "2022-10-22 22:50:28.562000"
test_bm = bm_football.filter((bm_football["start"] <= MAX_START) &
                             (bm_football["end"] >= MIN_END))
test_bm = test_bm\
    .withColumn('point_type', regexp_replace('point_type', 'goal', 'Goal'))\
    .withColumn('point_type', regexp_replace('point_type', 'corner', 'Corner'))\
    .withColumn('point_type', regexp_replace('point_type', 'red card', 'RedCard'))\
    .select('contest_id', 'section', col('point_type').alias('point'), col('market_type').alias('market'), 'away', 'start', 'end', 'home', 'league', 'region', 'sport', col('odds').alias('placement_odds'))

test_bm.write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/test_betmarkets_grouped")

time: 18.1 s (started: 2023-05-10 12:24:37 +02:00)


In [2]:
bm = spark.read.format("delta").load("/Users/andreikirpichev/parquet/betmarkets_lifespan/")
bm_by_outcome = bm.select('contest_id', 'section', 'point_type', 'market_type', 'market_params', 'away', 'end', 'home', 'league', col('odds').alias("odds_map"), 'region', 'sport', 'start', explode("odds").alias("outcome", "odds"))
bm_contest = bm_by_outcome.withColumn('contest_id', split(bm_by_outcome.contest_id, '\|').getItem(1))
bm_football = bm_contest.filter(bm_contest["sport"] == "Football")
# test: from 2022-10-18 09:32:32.232000  to  2022-10-28 14:23:44.972000 (974330, 13)
MAX_START = "2022-10-28 14:23:42.051000"
MIN_END = "2022-10-22 22:50:28.562000"
test_bm = bm_football.filter((bm_football["start"] <= MAX_START) &
                             (bm_football["end"] >= MIN_END))
test_bm = test_bm\
    .withColumn('point_type', regexp_replace('point_type', 'goal', 'Goal'))\
    .withColumn('point_type', regexp_replace('point_type', 'corner', 'Corner'))\
    .withColumn('point_type', regexp_replace('point_type', 'red card', 'RedCard'))\
    .select('contest_id', 'section', col('point_type').alias('point'), col('market_type').alias('market'), 'away', 'start', 'end', 'home', 'league', 'region', 'sport', col('odds').alias('placement_odds'))

#test_bm.write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/test_betmarkets")

time: 16.7 s (started: 2023-05-09 23:27:20 +02:00)


In [7]:
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/test_betmarkets_grouped/")
fixtures_df.show(5)

+----------+-------+-----+----------------+------------+--------------------+-------------------+---------------+----------------+------+--------+--------------------+
|contest_id|section|point|          market|        away|               start|                end|           home|          league|region|   sport|      placement_odds|
+----------+-------+-----+----------------+------------+--------------------+-------------------+---------------+----------------+------+--------+--------------------+
| FB-216767|     RT| Goal|BothTeamsToScore|UE Engordany|2022-10-22 16:53:...|2022-10-23 18:30:00|UE Santa Coloma|Primera Division|    AD|Football|{Yes -> 1.72, No ...|
| FB-216767|     RT| Goal|    CorrectScore|UE Engordany|2022-10-22 16:53:...|2022-10-23 18:30:00|UE Santa Coloma|Primera Division|    AD|Football|{ResultOutcome ->...|
| FB-216767|     RT| Goal|       EvenScore|UE Engordany|2022-10-22 18:19:...|2022-10-23 18:30:00|UE Santa Coloma|Primera Division|    AD|Football|{Yes -> 1.78, 

In [6]:
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/test_betmarkets_grouped/")
fixtures_df.groupby('section', 'point', 'market', 'league', 'region').(5)

AttributeError: 'GroupedData' object has no attribute 'show'

time: 149 ms (started: 2023-05-10 12:45:19 +02:00)


In [6]:
spark.sql("SELECT sequence(to_date('2018-01-01'), to_date('2018-03-01'), interval 1 month) as date").head()

Row(date=[datetime.date(2018, 1, 1), datetime.date(2018, 2, 1), datetime.date(2018, 3, 1)])

time: 61.4 ms (started: 2023-05-08 11:53:07 +02:00)


In [3]:
bets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/bet_stats/")
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/fixtures/")
customers_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/customers/")

recent_dates_pre = [
    #'2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17',
    '2022-09-18', '2022-09-19', '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24', '2022-09-25', '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30', '2022-09-31',
    '2022-10-01', '2022-10-02', '2022-10-03', '2022-10-04', '2022-10-05', '2022-10-06', '2022-10-07', '2022-10-08', '2022-10-09', '2022-10-10', '2022-10-11', '2022-10-12', '2022-10-13', '2022-10-14','2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-22', '2022-10-23', '2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-29', '2022-10-30']

recent_dates_pre.sort()
recent_dates = recent_dates_pre[14:] # drop first 14

bets_df = bets_df.filter(col("placement_date").isin(recent_dates_pre))\
                 .withColumn('contest_id', split('contest_id', '\|').getItem(1))

bf = bets_df.join(fixtures_df, "contest_id", "left")
bf_football = bf.filter(col("sport") == "Football")
#bf_football = bf_football.filter((col("customer_id") == "10047010") | (col("customer_id") == "10098010"))
data = bf_football.join(customers_df, bf_football.customer_id == customers_df.id, "left")

columns = ['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'placement_date', 'placement_time', 'placement_odds', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'stake_eur']
# 'bet_no', 'pick_index', 'fees', 'outcomes', 'currency', 'stake', 'placement_time', 'betting_types', 'scheduled_begin', 'birthdate'
c = CurrencyConverter(ECB_URL, fallback_on_missing_rate=True)
convert_curr = udf(lambda v,f,d : c.convert(v, f, 'EUR', d), DoubleType())

weekday_until_now = Window.partitionBy("customer_id", "dayofweek").rowsBetween(Window.unboundedPreceding, Window.currentRow)
minus2w_in_seconds = -60 * 60 * 24 * 14
section_last2weeks = Window.partitionBy("customer_id", "section").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)
#.orderBy(expr("unix_date(placement_date)").asc()).rangeBetween(-14, 0)
point_last2weeks = Window.partitionBy("customer_id", "point").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)
market_last2weeks = Window.partitionBy("customer_id", "market").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)
region_last2weeks = Window.partitionBy("customer_id", "region").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)
league_last2weeks = Window.partitionBy("customer_id", "league").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)

res = data.withColumn('age', ceil(datediff('placement_date', 'birthdate')/365.25))\
    .withColumn('minutes_before_start',
                  when(
                      round((unix_timestamp('scheduled_begin') - unix_timestamp('placement_time')) / 60) < 0,
                      0)
                  .otherwise(round((unix_timestamp('scheduled_begin') - unix_timestamp('placement_time')) / 60)))\
    .withColumn('section', split(bets_df['betting_types'].getItem(0), ' ').getItem(0))\
    .withColumn('point', split(bets_df['betting_types'].getItem(0), ' ').getItem(1))\
    .withColumn('market', split(bets_df['betting_types'].getItem(0), ' ').getItem(2))\
    .withColumn('market_params', regexp_extract(bets_df['betting_types'].getItem(1), '\(.*?\)', 0))\
    .withColumn("league", regexp_replace(col("league"), "[ .,;{}()\n\t=]", ""))\
    .filter(data['currency'] != 'NEO')\
    .withColumn('stake_eur', round(convert_curr('stake', 'currency', 'placement_date')/100, 2))\
    .select(*columns)\
    .withColumn("betmarket_id", concat_ws('_', col("section"), col("market"), col("point")))\
    .withColumn("match_id", concat_ws('_', col("league"), col("region")))\
    .withColumn("dayofweek", dayofweek(col("placement_date")))\
    .withColumn("2w_section", round(avg(col("stake_eur")).over(section_last2weeks), 2))\
    .withColumn("2w_point", round(avg(col("stake_eur")).over(point_last2weeks), 2))\
    .withColumn("2w_market", round(avg(col("stake_eur")).over(market_last2weeks), 2))\
    .withColumn("2w_region", round(avg(col("stake_eur")).over(region_last2weeks), 2))\
    .withColumn("2w_league", round(avg(col("stake_eur")).over(league_last2weeks), 2))\
    .withColumn("weekday_stake", round(avg(col("stake_eur")).over(weekday_until_now), 2))\
    .filter(col("age").isNotNull() & col("country").isNotNull())\
    .orderBy(col("placement_time").asc())


#.withColumn("section_2w", round(avg(col("stake_eur")).over(section_last2weeks), 2))\
#.withColumn("point_2w", round(avg(col("stake_eur")).over(point_last2weeks), 2))\
#.withColumn("market_2w", round(avg(col("stake_eur")).over(market_last2weeks), 2))\

time: 1.23 s (started: 2023-05-11 09:33:26 +02:00)


In [5]:
res.filter(col("placement_date").isin(recent_dates))\
    .write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/bets_rl")

23/05/10 17:23:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


time: 1min 37s (started: 2023-05-10 17:23:19 +02:00)


In [1]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

main = DeltaTable("/Users/andreikirpichev/parquet/bets_oct_pers_backup/").to_pandas()
rl = DeltaTable("/Users/andreikirpichev/parquet/bets_rl/").to_pandas()
main.sort_values('placement_time', ascending=False, inplace=True)
rl.sort_values('placement_time', ascending=False, inplace=True)
main.reset_index(drop=True, inplace=True)
rl.reset_index(drop=True, inplace=True)
#

# extracted_col = rl["2w_section"]
# main.insert(1, "2w_section_check", extracted_col)
main[["2w_region", "2w_league", "2w_section_check"]] = rl[["2w_region", "2w_league", "2w_section"]]
#spark.createDataFrame(main).write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/bets_oct_pers")

In [5]:
main.to_parquet("/Users/andreikirpichev/parquet/bets_oct_pers.parquet")

time: 1min 26s (started: 2023-05-10 19:20:13 +02:00)


In [3]:
maindf = spark.createDataFrame(main)
maindf.show(3)

KeyboardInterrupt: 

time: 18min 25s (started: 2023-05-10 18:34:08 +02:00)


In [ ]:
maindf.write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/bets_oct_pers")

In [5]:
extracted_col = rl["2w_section"]
main.insert(1, "2w_section_check", extracted_col)

In [6]:
rl.head(5)

,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,placement_odds,home,away,region,sport,league,minutes_before_start,stake_eur,betmarket_id,match_id,dayofweek,2w_section,2w_point,2w_market,2w_region,2w_league,weekday_stake
0,3044310,40,AT,HT1,Goal,OverUnder,None,FB-226375,2022-10-28,2022-10-28 14:23:42.051,1.82,FC Ma'an,Shabab Al Ordon,JO,Football,PremierLeague,0.0,10.86,HT1_OverUnder_Goal,PremierLeague_JO,6,21.41,20.25,20.70,13.72,20.43,20.66
1,17707010,60,DE,RT,Goal,OverUnder,None,FB-226334,2022-10-28,2022-10-28 14:23:40.631,1.90,El Sekka El Hadid,Eastern Company SC,EG,Football,2ndDivision,0.0,20.00,RT_OverUnder_Goal,2ndDivision_EG,6,12.84,13.01,13.09,20.63,22.50,21.67
2,11764610,61,DE,RT,Goal,Handicap3w,None,FB-226449,2022-10-28,2022-10-28 14:23:39.479,1.66,Arda Kardzhali,CSKA Sofia,BG,Football,FirstLeague,66.0,0.50,RT_Handicap3w_Goal,FirstLeague_BG,6,0.27,0.27,0.27,0.50,0.50,0.24
3,11764610,61,DE,RT,Goal,Handicap3w,None,FB-219054,2022-10-28,2022-10-28 14:23:39.479,2.08,SpVgg Greuther Fürth,Arminia Bielefeld,DE,Football,2Bundesliga,126.0,0.50,RT_Handicap3w_Goal,2Bundesliga_DE,6,0.27,0.27,0.27,0.38,0.37,0.25
4,11764610,61,DE,RT,Goal,Handicap3w,None,FB-219064,2022-10-28,2022-10-28 14:23:39.479,2.86,1. FC Magdeburg,1. FC Heidenheim,DE,Football,2Bundesliga,126.0,0.50,RT_Handicap3w_Goal,2Bundesliga_DE,6,0.27,0.27,0.27,0.38,0.37,0.24


In [7]:
main.head(5)

,customer_id,2w_section_check,age,country,section,point,market,contest_id,placement_date,placement_time,placement_odds,home,away,region,sport,league,minutes_before_start,stake_eur,betmarket_id,match_id,dayofweek,2w_section,2w_point,2w_market,weekday_stake,recent1_betmarket,recent1_match,recent1_section,recent1_market,recent1_point,recent1_league,recent1_region,recent2_betmarket,recent2_match,recent2_section,recent2_market,recent2_point,recent2_league,recent2_region,recent3_betmarket,recent3_match,recent3_section,recent3_market,recent3_point,recent3_league,recent3_region,recent4_betmarket,recent4_match,recent4_section,recent4_market,recent4_point,recent4_league,recent4_region,recent5_betmarket,recent5_match,recent5_section,recent5_market,recent5_point,recent5_league,recent5_region,recent6_betmarket,recent6_match,recent6_section,recent6_market,recent6_point,recent6_league,recent6_region,recent7_betmarket,recent7_match,recent7_section,recent7_market,recent7_point,recent7_league,recent7_region,recent8_betmarket,recent8_match,recent8_section,recent8_market,recent8_point,recent8_league,recent8_region,recent9_betmarket,recent9_match,recent9_section,recent9_market,recent9_point,recent9_league,recent9_region,recent10_betmarket,recent10_match,recent10_section,recent10_market,recent10_point,recent10_league,recent10_region,pers_section_RT,pers_section_HT1,pers_section_HT2,pers_section_OT,pers_section_LT,pers_section_RTOT,pers_section_SO,pers_section_OT1,pers_section_OT2,pers_section_FT,pers_market_OverUnder,pers_market_Handicap3w,pers_market_BothTeamsToScore,pers_market_NthPoint,pers_market_TeamOverUnder,pers_market_PlayerToScore,pers_market_TeamToScore,pers_market_CorrectScore,pers_market_Handicap2w,pers_market_ToScoreEachSection,pers_point_Goal,pers_point_Corner,pers_point_RedCard,pers_league_UEFAChampionsLeague,pers_league_Bundesliga,pers_league_PremierLeague,pers_league_UEFANationsLeague,pers_league_UEFAEuropaLeague,pers_league_SerieA,pers_league_LaLiga,pers_league_2Bundesliga,pers_league_UEFAEuropaConferenceLeague,pers_league_Ligue1,pers_league_Cup,pers_league_PrimeraDivision,pers_league_DFBPokal,pers_league_BattleEsoccer8Min,pers_league_3Liga,pers_league_SuperLeague,pers_league_LigaProfesional,pers_league_JupilerProLeague,pers_league_SuperLig,pers_league_Championship,pers_league_UEFAYouthLeague,pers_league_PrimeiraLiga,pers_league_SerieB,pers_league_Eredivisie,pers_league_LeagueOne,pers_league_LaLiga2,pers_league_ReserveLeague,pers_league_LiveArenaEsoccer10Min,pers_league_EersteDivisie,pers_league_1stDivision,pers_region_EUR,pers_region_DE,pers_region_ENG,pers_region_ES,pers_region_IT,pers_region_FR,pers_region_AR,pers_region_ONL,pers_region_NL,pers_region_TR,pers_region_BE,pers_region_BR,pers_region_PT,pers_region_MX,pers_region_JP,pers_region_CN,pers_region_INT,pers_region_IL,pers_region_NO,pers_region_AT,pers_region_DK,pers_region_CO,pers_region_SE,pers_region_KR,pers_region_CH,pers_region_CZ,pers_region_RO,pers_region_PY,pers_region_US,pers_region_FI,persTOP1_betmarket,persTOP1_match,persTOP1_section,persTOP1_market,persTOP1_point,persTOP1_league,persTOP1_region,persTOP2_betmarket,persTOP2_match,persTOP2_section,persTOP2_market,persTOP2_point,persTOP2_league,persTOP2_region,persTOP3_betmarket,persTOP3_match,persTOP3_section,persTOP3_market,persTOP3_point,persTOP3_league,persTOP3_region,persTOP4_betmarket,persTOP4_match,persTOP4_section,persTOP4_market,persTOP4_point,persTOP4_league,persTOP4_region,persTOP5_betmarket,persTOP5_match,persTOP5_section,persTOP5_market,persTOP5_point,persTOP5_league,persTOP5_region,persTOP6_betmarket,persTOP6_match,persTOP6_section,persTOP6_market,persTOP6_point,persTOP6_league,persTOP6_region,persTOP7_betmarket,persTOP7_match,persTOP7_section,persTOP7_market,persTOP7_point,persTOP7_league,persTOP7_region,persTOP8_betmarket,persTOP8_match,persTOP8_section,persTOP8_market,persTOP8_point,persTOP8_league,persTOP8_region,persTOP9_betmarket,persTOP9_match,persTOP9_section,persTOP9_market,persTOP9_point,per

In [9]:
main["checksum"] = main["2w_section_check"] - main["2w_section"]
print(main["2w_section_check"].sum(), main["2w_section"].sum())

12857822.99999999 12857822.99999999


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#res.toPandas().head(10000)

In [4]:
w = Window.partitionBy('customer_id').orderBy(col('placement_time').asc())

time: 9.02 ms (started: 2023-05-11 09:33:44 +02:00)


In [5]:
previous = w.rowsBetween(Window.unboundedPreceding, -1)
previous_incl_self = w.rowsBetween(Window.unboundedPreceding, 0)
res = res\
    .withColumn("recent1_betmarket", when(lag("betmarket_id", 1).over(w) != col("betmarket_id"),
                                          lag("betmarket_id", 1).over(w)))\
    .withColumn("recent1_match", when(lag("match_id", 1).over(w) != col("match_id"),
                                          lag("match_id", 1).over(w)))

for i in range (1, 11):
    recentN_betmarket = "recent{}_betmarket".format(i)
    recentN_match = "recent{}_match".format(i)
    if i > 1:
        res = res.withColumn(recentN_betmarket, when(lag("betmarket_id", 1).over(w) != col("betmarket_id"),
                                       last(col("recent{}_betmarket".format(i - 1)), True).over(previous)))\
                 .withColumn(recentN_match, when(lag("match_id", 1).over(w) != col("match_id"),
                                       last(col("recent{}_match".format(i - 1)), True).over(previous)))

    res = res.withColumn(recentN_betmarket, last(col(recentN_betmarket), True).over(previous_incl_self))\
             .withColumn(recentN_match, last(col(recentN_match), True).over(previous_incl_self))\
             .withColumn("recent{}_section".format(i), split(recentN_betmarket, '_').getItem(0))\
             .withColumn("recent{}_market".format(i), split(recentN_betmarket, '_').getItem(1))\
             .withColumn("recent{}_point".format(i), split(recentN_betmarket, '_').getItem(2))\
             .withColumn("recent{}_league".format(i), split(recentN_match, '_').getItem(0))\
             .withColumn("recent{}_region".format(i), split(recentN_match, '_').getItem(1))

time: 2.16 s (started: 2023-05-11 09:33:50 +02:00)


In [6]:
ohf = {
    "section": 10,
    "market": 10,
    "point": 10,
    "league": 30,
    "region": 30
}
for feat, n in ohf.items():
    types_bets = res.groupBy(feat).count().orderBy(col('count').desc()).limit(n).rdd.map(lambda x: x[0]).collect()
    types_expr = [when(col(feat) == ty, col("stake_eur")).otherwise(0).alias(feat + '_' + ty) for ty in types_bets]
    res = res.select(col('*'), *types_expr)

23/05/11 09:33:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


time: 59 s (started: 2023-05-11 09:33:54 +02:00)


In [7]:
keys = list(ohf.keys())
headers = [
    [col for col in res.columns
     if col.startswith(header + '_') &
     (col != "market_params")
     ] for header in keys
]

def flatten(l):
    return [item for sublist in l for item in sublist]

segment = 'pers'
minus2w_in_seconds = -60 * 60 * 24 * 14
over_last2weeks_pers = Window.partitionBy("customer_id").orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)

for header in flatten(headers):

    sum_col = header + '_sum_{}' + segment
    count_col = header + '_count_{}' + segment
    stake_col = segment + '_' + header
    res = res\
        .withColumn(sum_col, sum(when(col(header) > 0, col("stake_eur")).otherwise(0)).over(over_last2weeks_pers))\
        .withColumn(count_col, sum(when(col(header) > 0, 1).otherwise(0)).over(over_last2weeks_pers))\
        .withColumn(stake_col, round(when(col(count_col) > 0,
                                          col(sum_col) / col(count_col))
                                     .otherwise(0), 2))\
        .drop(sum_col, count_col)

time: 1min 11s (started: 2023-05-11 09:34:53 +02:00)


In [4]:
keys = list(ohf.keys())
headers = [
    [col for col in res.columns
     if col.startswith(header + '_') &
     (col != "market_params")
     ] for header in keys
]

def flatten(l):
    return [item for sublist in l for item in sublist]

segment = 'glob'
minus2w_in_seconds = -60 * 60 * 24 * 14
over_last2weeks_glob = Window.orderBy(expr("unix_timestamp(placement_time)").asc()).rangeBetween(minus2w_in_seconds, 0)

for header in flatten(headers):

    sum_col = header + '_sum_{}' + segment
    count_col = header + '_count_{}' + segment
    stake_col = segment + '_' + header
    res = res\
        .withColumn(sum_col, sum(when(col(header) > 0, col("stake_eur")).otherwise(0)).over(over_last2weeks_glob))\
        .withColumn(count_col, sum(when(col(header) > 0, 1).otherwise(0)).over(over_last2weeks_glob))\
        .withColumn(stake_col, round(when(col(count_col) > 0,
                                          col(sum_col) / col(count_col))
                                    .otherwise(0), 2))\
        .drop(sum_col, count_col)

time: 38.7 s (started: 2023-05-10 22:30:18 +02:00)


In [8]:
import numpy as np

segment = 'pers'

betmarkets = headers[0:3]
matches = headers[3:5]

betmarket_id = 'betmarketID' + segment
match_id = 'matchID' + segment

betmarket_ids = np.array(
    np.meshgrid(*betmarkets)).T.reshape(-1, 3)

match_ids = np.array(
    np.meshgrid(*matches)).T.reshape(-1, 2)

for betmarket in betmarket_ids:
    stakes_sum = np.sum([col(segment + '_' + ty) for ty in betmarket])
    res = res.withColumn(betmarket_id + '_'.join(betmarket), stakes_sum)

for match in match_ids:
    stakes_sum = np.sum([col(segment + '_' + ty) for ty in match])
    res = res.withColumn(match_id + '_'.join(match), stakes_sum)

betmarket_stakes = sort_array(array(*[
    struct(col(c).alias("v"), lit(c).alias("k")) for c in res.columns if c.startswith(betmarket_id)
]), asc=False)
match_stakes = sort_array(array(*[
    struct(col(c).alias("v"), lit(c).alias("k")) for c in res.columns if c.startswith(match_id)
]), asc=False)

for i in range(0, 10):
    res = res\
        .select(col("*"),
                     *[betmarket_stakes[i]["k"].alias("{}TOP{}_betmarket".format(segment, i+1))],
                     *[match_stakes[i]["k"].alias("{}TOP{}_match".format(segment, i+1))])\
        .withColumn("{}TOP{}_section".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(1))\
        .withColumn("{}TOP{}_market".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(3))\
        .withColumn("{}TOP{}_point".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(5))\
        .withColumn("{}TOP{}_league".format(segment, i+1), split("{}TOP{}_match".format(segment, i+1), '_').getItem(1))\
        .withColumn("{}TOP{}_region".format(segment, i+1), split("{}TOP{}_match".format(segment, i+1), '_').getItem(3))

time: 23min 47s (started: 2023-05-11 09:36:05 +02:00)


In [5]:
import numpy as np

segment = 'glob'

betmarkets = headers[0:3]
matches = headers[3:5]

betmarket_id = 'betmarketID' + segment
match_id = 'matchID' + segment

betmarket_ids = np.array(
    np.meshgrid(*betmarkets)).T.reshape(-1, 3)

match_ids = np.array(
    np.meshgrid(*matches)).T.reshape(-1, 2)

for betmarket in betmarket_ids:
    stakes_sum = np.sum([col(segment + '_' + ty) for ty in betmarket])
    res = res.withColumn(betmarket_id + '_'.join(betmarket), stakes_sum)

for match in match_ids:
    stakes_sum = np.sum([col(segment + '_' + ty) for ty in match])
    res = res.withColumn(match_id + '_'.join(match), stakes_sum)

betmarket_stakes = sort_array(array(*[
    struct(col(c).alias("v"), lit(c).alias("k")) for c in res.columns if c.startswith(betmarket_id)
]), asc=False)
match_stakes = sort_array(array(*[
    struct(col(c).alias("v"), lit(c).alias("k")) for c in res.columns if c.startswith(match_id)
]), asc=False)

for i in range(0, 10):
    res = res\
        .select(col("*"),
                   *[betmarket_stakes[i]["k"].alias("{}TOP{}_betmarket".format(segment, i+1))],
                   *[match_stakes[i]["k"].alias("{}TOP{}_match".format(segment, i+1))])\
        .withColumn("{}TOP{}_section".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(1))\
        .withColumn("{}TOP{}_market".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(3))\
        .withColumn("{}TOP{}_point".format(segment, i+1), split("{}TOP{}_betmarket".format(segment, i+1), '_').getItem(5))\
        .withColumn("{}TOP{}_league".format(segment, i+1), split("{}TOP{}_match".format(segment, i+1), '_').getItem(1))\
        .withColumn("{}TOP{}_region".format(segment, i+1), split("{}TOP{}_match".format(segment, i+1), '_').getItem(3))

time: 18min 28s (started: 2023-05-10 22:30:57 +02:00)


In [4]:
res.toPandas().memory_usage()

23/05/08 22:00:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 9]
Traceback (most recent call last):
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

time: 1min 31s (started: 2023-05-08 22:00:49 +02:00)


In [9]:
res = res.drop(*[c for c in res.columns if c.startswith("matchID")
                 | c.startswith("betmarketID")
                 | c.startswith("section_")
                 | c.startswith("point_")
                 | c.startswith("market_")
                 | c.startswith("league_")
                 | c.startswith("region_")
                 ])\
          .filter(col("placement_date").isin(recent_dates))

time: 5.97 s (started: 2023-05-11 09:59:52 +02:00)


In [10]:
res.write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/bets_oct_pers")

23/05/11 10:15:17 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB


time: 2h 29min 16s (started: 2023-05-11 09:59:58 +02:00)


In [9]:
cols = [c for c in res.columns if c.startswith("pers_market")]
res.toPandas()[cols]

23/05/09 11:25:45 WARN DAGScheduler: Broadcasting large task binary with size 1575.1 KiB


,pers_market_OverUnder,pers_market_Handicap3w,pers_market_NumberOfPoints3w,pers_market_OverUnderInTime
0,96.00,0.00,0.0,0.0
1,34.33,0.00,0.0,0.0
2,34.33,0.00,0.0,0.0
3,34.33,0.00,25.0,0.0
4,26.00,0.00,25.0,0.0
5,21.80,0.00,25.0,0.0
6,19.00,0.00,25.0,0.0
7,16.75,0.00,25.0,0.0
8,16.75,0.00,25.0,0.0
9,16.00,0.00,25.0,0.0


time: 4.76 s (started: 2023-05-09 11:25:45 +02:00)


In [ ]:
types_bets = res.groupBy("league").count().orderBy(col('count').desc()).limit(30).show()

In [8]:
res.write.format("delta").mode("append").save("/Users/andreikirpichev/parquet/bets_sm1")

In [4]:
bets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/bets/")

bets_df.select(countDistinct("market_params")).show()
bets_df.select([count(when(col(c).isNull(), c)).alias(c) for c in bets_df.columns]).show()

+-----------------------------+
|count(DISTINCT market_params)|
+-----------------------------+
|                         1306|
+-----------------------------+



+-----------+---+-------+-------+-----+------+-------------+----------+--------------+--------------+--------------+----+----+------+-----+------+--------------------+---------+---------+-------------+----------+--------+---------+
|customer_id|age|country|section|point|market|market_params|contest_id|placement_date|placement_time|placement_odds|home|away|region|sport|league|minutes_before_start|stake_eur|dayofweek|weekday_stake|section_2w|point_2w|market_2w|
+-----------+---+-------+-------+-----+------+-------------+----------+--------------+--------------+--------------+----+----+------+-----+------+--------------------+---------+---------+-------------+----------+--------+---------+
|          0|  0|      0|      0|    0|     0|      3752896|         0|             0|             0|             0|   0|   0|     0|    0|     0|                   0|        0|        0|            0|         0|       0|        0|
+-----------+---+-------+-------+-----+------+-------------+----------+-